In [0]:

import psycopg2
import datetime
spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
import aaplproxy
from aadatapipelinecore.core.urn import Urn
from aaplproxy.da.local_sqlrunner import LocalSqlRunner
from aadatapipelinecore.core.utils.module import application_settings
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql import functions as F
ios_feed = {1: "0,1,2,100,101,102"}
android_feed = {0: "0,1,2"}
def citus_row(start,end):
    def get_data_in_citus(start,end):
        citus_dsn_ = (
            "dbname='{db}' user='{user}' password='{password}' "
            "host='{host}' port='{port}'".format(
                db="aa_store_db",
                user="citus_bdp_prod_app_int_qa",
                host="10.2.6.141",
                password="wZw8cfBuuklIskVG",
                port=5432
            )
        )
        sql = "select device_code, cast(sum(est_free_app_download) as bigint) as est_free_app_download, cast(sum(est_paid_app_download) as bigint) as est_paid_app_download, cast(sum(est_revenue) as bigint) as est_revenue from store.store_est_publisher_fact_v2 where date>='{}' and date<='{}' group by device_code".format(start,end)
        db_data = query(citus_dsn_, sql)
        return db_data
    def query(dsn, sql):
        with psycopg2.connect(dsn) as conn:
            conn.autocommit = True
            with conn.cursor() as cur:
                cur.execute(sql)
                result = cur.fetchall()
                conn.commit()
        return result
    result = get_data_in_citus(start, end)
    return [Row(device_code=r[0], sum_est_free_app_download=r[1], sum_est_paid_app_download=r[2],
                sum_est_revenue=r[3]) for r in result]
def generate_citus_result(spark, citus_data):
    schema = StructType([
    StructField("device_code",  StringType(), True),
    StructField("sum_est_free_app_download", LongType(), True),
    StructField("sum_est_paid_app_download", LongType(), True),
    StructField("sum_est_revenue", LongType(), True)])
    df_3 = spark.createDataFrame(citus_data, schema)
    df_3.createOrReplaceTempView("citus_data")
    return spark.sql("select * from citus_data order by device_code desc")
print("=============Online============")
online_data_df = generate_citus_result(spark,citus_row("2020-08-01","2020-08-01"))
online_data_df.show()
print("=============sampleData============")
sample_data_df = spark.read.format("parquet").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.publisher-download-revenue.v1/fact/")
sample_data_df.createOrReplaceTempView("sample_data")
result_df = spark.sql("""
     select device_code,
            sum(est_free_app_download) as sum_est_free_app_download,
            sum(est_paid_app_download) as sum_est_paid_app_download,
            sum(est_revenue) as sum_est_revenue
    from sample_data
    group by device_code
""")

print(sample_data_df.count())
online_data_df.exceptAll(result_df).show()
result_df.exceptAll(online_data_df).show()

In [0]:

import psycopg2
import datetime
spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
import aaplproxy
from aadatapipelinecore.core.urn import Urn
from aaplproxy.da.local_sqlrunner import LocalSqlRunner
from aadatapipelinecore.core.utils.module import application_settings
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql import functions as F
ios_feed = {1: "0,1,2,100,101,102"}
android_feed = {0: "0,1,2"}
def citus_row(start,end):
    def get_data_in_citus(start,end):
        citus_dsn_ = (
            "dbname='{db}' user='{user}' password='{password}' "
            "host='{host}' port='{port}'".format(
                db="aa_store_db",
                user="citus_bdp_prod_app_int_qa",
                host="10.2.6.141",
                password="wZw8cfBuuklIskVG",
                port=5432
            )
        )
        sql = "select publisher_id, cast(sum(est_free_app_download) as bigint) as est_free_app_download, cast(sum(est_paid_app_download) as bigint) as est_paid_app_download, cast(sum(est_revenue) as bigint) as est_revenue from store.store_est_publisher_fact_v2 where date>='{}' and date<='{}' group by publisher_id".format(start,end)
        db_data = query(citus_dsn_, sql)
        return db_data
    def query(dsn, sql):
        with psycopg2.connect(dsn) as conn:
            conn.autocommit = True
            with conn.cursor() as cur:
                cur.execute(sql)
                result = cur.fetchall()
                conn.commit()
        return result
    result = get_data_in_citus(start, end)
    return [Row(publisher_id=r[0], sum_est_free_app_download=r[1], sum_est_paid_app_download=r[2],
                sum_est_revenue=r[3]) for r in result]
def generate_citus_result(spark, citus_data):
    schema = StructType([
    StructField("publisher_id",  LongType(), True),
    StructField("sum_est_free_app_download", LongType(), True),
    StructField("sum_est_paid_app_download", LongType(), True),
    StructField("sum_est_revenue", LongType(), True)])
    df_3 = spark.createDataFrame(citus_data, schema)
    df_3.createOrReplaceTempView("citus_data")
    return spark.sql("select * from citus_data order by publisher_id desc")
print("=============Online============")
online_data_df = generate_citus_result(spark,citus_row("2020-08-01","2020-08-01"))
print("=============sampleData============")
sample_data_df = spark.read.format("parquet").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.publisher-download-revenue.v1/fact/")
sample_data_df.createOrReplaceTempView("sample_data")
result_df = spark.sql("""
     select publisher_key,
            sum(est_free_app_download) as sum_est_free_app_download,
            sum(est_paid_app_download) as sum_est_paid_app_download,
            sum(est_revenue) as sum_est_revenue
    from sample_data
    where device_code = 'android-all' or device_code = 'ios-tablet' or device_code = 'ios-phone' 
    group by publisher_key
""")

print(sample_data_df.count())
online_data_df.exceptAll(result_df).show()
result_df.exceptAll(online_data_df).show()

In [0]:

result_df.where("publisher_key = 20200003671279").show()

In [0]:

raw_data = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-publisher-dna-log.v1/fact/").where(" granularity='daily' and date = '2020-08-01'")
sample_data = spark.read.format("parquet").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.publisher-download-revenue.v1/fact/")
raw_data.createOrReplaceTempView("raw_data")
sample_data.createOrReplaceTempView("sample_data")

print("result_view")
result1_df = spark.sql("""
    select device_code,
           sum(est_paid_app_download) as sum_est_paid_app_download,
           sum(est_free_app_download) as sum_est_free_app_download,
           sum(est_in_app_purchase_revenue) as sum_est_in_app_purchase_revenue,
           sum(est_non_in_app_purchase_revenue) as sum_est_non_in_app_purchase_revenue,
           sum(est_revenue) as sum_est_revenue
    from sample_data
    group by device_code
""")
print("result_data")
result1_df.show()
spark.sql("""
    select sum(coalesce(est_free_app_download, 0) + coalesce(est_paid_app_download, 0)) as sum_est_paid_app_download,
           sum(est_download)
    from sample_data
""").show()
print("raw_data")
raw1_df = spark.sql("""
    select device_code,
           sum(est_paid_app_download) as sum_est_paid_app_download,
           sum(est_free_app_download) as sum_est_free_app_download,
           sum(est_revenue_iap) as sum_est_in_app_purchase_revenue,
           sum(est_revenue_non_iap) as sum_est_non_in_app_purchase_revenue,
           sum(est_revenue) as sum_est_revenue
    from raw_data
    group by device_code
""")
print("raw_data")
raw1_df.show()
print("exceptAll ")
raw1_df.exceptAll(result1_df).show()
result1_df.exceptAll(raw1_df).show()
print("sample_data_count")
spark.sql("""
    select device_code, count(1) as count
    from sample_data
    group by device_code
""").show()
print("raw_data_count")
spark.sql("""
    select device_code, count(1) as count
    from raw_data
    group by device_code
""").show()


In [0]:
%%sh
PGPASSWORD='wZw8cfBuuklIskVG' psql -h 10.2.6.141 -U citus_bdp_prod_app_int_qa -d aa_store_db -p 5432 << EOF 
SELECT * from store.store_est_publisher_fact_v2 where publisher_id=20200003671279 and date='2020-08-01' limit 1;
EOF


In [0]:

import psycopg2
import datetime
spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
import aaplproxy
from aadatapipelinecore.core.urn import Urn
from aaplproxy.da.local_sqlrunner import LocalSqlRunner
from aadatapipelinecore.core.utils.module import application_settings
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql import functions as F
ios_feed = {1: "0,1,2,100,101,102"}
android_feed = {0: "0,1,2"}
def citus_row(start,end):
    def get_data_in_citus(start,end):
        citus_dsn_ = (
            "dbname='{db}' user='{user}' password='{password}' "
            "host='{host}' port='{port}'".format(
                db="aa_store_db",
                user="citus_bdp_prod_app_int_qa",
                host="10.2.6.141",
                password="wZw8cfBuuklIskVG",
                port=5432
            )
        )
        sql = "select country_code, cast(sum(est_free_app_download) as bigint) as est_free_app_download, cast(sum(est_paid_app_download) as bigint) as est_paid_app_download, cast(sum(est_revenue) as bigint) as est_revenue from store.store_est_publisher_fact_v2 where date>='{}' and date<='{}' group by country_code".format(start,end)
        db_data = query(citus_dsn_, sql)
        return db_data
    def query(dsn, sql):
        with psycopg2.connect(dsn) as conn:
            conn.autocommit = True
            with conn.cursor() as cur:
                cur.execute(sql)
                result = cur.fetchall()
                conn.commit()
        return result
    result = get_data_in_citus(start, end)
    return [Row(country_code=r[0], sum_est_free_app_download=r[1], sum_est_paid_app_download=r[2],
                sum_est_revenue=r[3]) for r in result]
def generate_citus_result(spark, citus_data):
    schema = StructType([
    StructField("country_code",  StringType(), True),
    StructField("sum_est_free_app_download", LongType(), True),
    StructField("sum_est_paid_app_download", LongType(), True),
    StructField("sum_est_revenue", LongType(), True)])
    df_3 = spark.createDataFrame(citus_data, schema)
    df_3.createOrReplaceTempView("citus_data")
    return spark.sql("select * from citus_data order by country_code desc")
print("=============Online============")
online_data_df = generate_citus_result(spark,citus_row("2020-08-01","2020-08-01"))
print("=============sampleData============")
sample_data_df = spark.read.format("parquet").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.publisher-download-revenue.v1/fact/")
sample_data_df.createOrReplaceTempView("sample_data")
result_df = spark.sql("""
     select country_code,
            sum(est_free_app_download) as sum_est_free_app_download,
            sum(est_paid_app_download) as sum_est_paid_app_download,
            sum(est_revenue) as sum_est_revenue
    from sample_data
    where device_code = 'android-all' or device_code = 'ios-tablet' or device_code = 'ios-phone' 
    group by country_code
""")

print(sample_data_df.count())
online_data_df.exceptAll(result_df).show()
result_df.exceptAll(online_data_df).show()

In [0]:

import psycopg2
import datetime
spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
import aaplproxy
from aadatapipelinecore.core.urn import Urn
from aaplproxy.da.local_sqlrunner import LocalSqlRunner
from aadatapipelinecore.core.utils.module import application_settings
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql import functions as F
ios_feed = {1: "0,1,2,100,101,102"}
android_feed = {0: "0,1,2"}
def citus_row(start,end):
    def get_data_in_citus(start,end):
        citus_dsn_ = (
            "dbname='{db}' user='{user}' password='{password}' "
            "host='{host}' port='{port}'".format(
                db="aa_store_db",
                user="citus_bdp_prod_app_int_qa",
                host="10.2.6.141",
                password="wZw8cfBuuklIskVG",
                port=5432
            )
        )
        sql = "select category_id, cast(sum(est_free_app_download) as bigint) as est_free_app_download, cast(sum(est_paid_app_download) as bigint) as est_paid_app_download, cast(sum(est_revenue) as bigint) as est_revenue from store.store_est_publisher_fact_v2 where date>='{}' and date<='{}' group by category_id".format(start,end)
        db_data = query(citus_dsn_, sql)
        return db_data
    def query(dsn, sql):
        with psycopg2.connect(dsn) as conn:
            conn.autocommit = True
            with conn.cursor() as cur:
                cur.execute(sql)
                result = cur.fetchall()
                conn.commit()
        return result
    result = get_data_in_citus(start, end)
    return [Row(category_id=r[0], sum_est_free_app_download=r[1], sum_est_paid_app_download=r[2],
                sum_est_revenue=r[3]) for r in result]
def generate_citus_result(spark, citus_data):
    schema = StructType([
    StructField("category_id",  StringType(), True),
    StructField("sum_est_free_app_download", LongType(), True),
    StructField("sum_est_paid_app_download", LongType(), True),
    StructField("sum_est_revenue", LongType(), True)])
    df_3 = spark.createDataFrame(citus_data, schema)
    df_3.createOrReplaceTempView("citus_data")
    return spark.sql("select * from citus_data order by category_id desc")
print("=============Online============")
online_data_df = generate_citus_result(spark,citus_row("2020-08-01","2020-08-01"))
print("=============sampleData============")
sample_data_df = spark.read.format("parquet").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.publisher-download-revenue.v1/fact/")
sample_data_df.createOrReplaceTempView("sample_data")
result_df = spark.sql("""
     select category_key,
            sum(est_free_app_download) as sum_est_free_app_download,
            sum(est_paid_app_download) as sum_est_paid_app_download,
            sum(est_revenue) as sum_est_revenue
    from sample_data
    where device_code = 'android-all' or device_code = 'ios-tablet' or device_code = 'ios-phone' 
    group by category_key
""")

print(sample_data_df.count())
online_data_df.exceptAll(result_df).show()
result_df.exceptAll(online_data_df).show()